# DREsS Dataset Pointwise Fine Tuned Model


In [ ]:
from google.colab import drive, userdata
drive.mount('/content/drive')

!pip install -q "transformers>=4.37" accelerate bitsandbytes datasets pandas tqdm peft sentencepiece

import os
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
from datasets import Dataset
import matplotlib.pyplot as plt

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer,
)

from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    PeftModel,
)

from huggingface_hub import login

HF_API_KEY = userdata.get("HF_API_KEY")
if HF_API_KEY is None:
    raise ValueError("Missing HF_API_KEY in Colab Secrets")
login(HF_API_KEY)

project_dir = "/content/drive/MyDrive/DREsS_Dataset_allocation_harms"
os.makedirs(project_dir, exist_ok=True)

print("Project directory:", project_dir)


In [ ]:
clean_full_path = f"{project_dir}/DRESs_cleaned_full.csv"
df = pd.read_csv(clean_full_path)
print(df.shape)

# Build training text
def build_sft_text(row):
    return (
        "You are an automatic writing scoring system. "
        "You MUST answer with ONLY a number between 1 and 5.\n\n"
        f"Essay prompt:\n{row['prompt']}\n\n"
        f"Essay:\n{row['essay']}\n\n"
        f"Final score: {row['total']}"
    )

df["text"] = df.apply(build_sft_text, axis=1)
df = df[["text"]].copy()

# Train/val split
df_train = df.sample(frac=0.2, random_state=42)
df_val = df.drop(df_train.index).reset_index(drop=True)
df_train = df_train.reset_index(drop=True)

print("Train:", df_train.shape, "Val:", df_val.shape)

In [ ]:
def tokenize_dataset(df, tokenizer, max_len=1024):
    ds = Dataset.from_pandas(df)

    def tok(batch):
        out = tokenizer(
            batch["text"],
            truncation=True,
            max_length=max_len,
            padding="max_length",
        )
        out["labels"] = out["input_ids"].copy()
        return out

    return ds.map(tok, batched=True, remove_columns=["text"])

In [ ]:
def run_qlora_finetune(
    model_name,
    output_dir,
    train_df,
    val_df,
    max_length=1024,
    num_epochs=2,
    lr=2e-4,
):

    print("Loading tokenizer and 4-bit model")

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )

    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
    )

    print("Preparing model for QLoRA training...")
    model = prepare_model_for_kbit_training(model)

    lora_cfg = LoraConfig(
        r=8,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "v_proj"]
    )

    model = get_peft_model(model, lora_cfg)
    model.print_trainable_parameters()

    print("\nTokenizing dataset")
    train_ds = tokenize_dataset(train_df, tokenizer, max_length)
    val_ds = tokenize_dataset(val_df, tokenizer, max_length)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )

    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=num_epochs,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=8,
        save_steps=200,
        logging_steps=50,
        learning_rate=lr,
        fp16=True,
        bf16=False,
        max_grad_norm=1.0,
        warmup_ratio=0.03,
        lr_scheduler_type="cosine",
        report_to="none",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        data_collator=data_collator,
    )

    print("\nStarting training")
    trainer.train()

    print("\nSaving adapter")
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    print("Saved LoRA adapter to:", output_dir)
    return output_dir


In [ ]:
llama_output = f"{project_dir}/llama2_dress_scoring_lora"

run_qlora_finetune(
    model_name="meta-llama/Llama-2-7b-chat-hf",
    output_dir=llama_output,
    train_df=df_train,
    val_df=df_val,
    max_length=1024,
    num_epochs=2,
    lr=2e-4,
)

In [ ]:
qwen_output = f"{project_dir}/qwen2_5_7b_dress_scoring_lora"

run_qlora_finetune(
    model_name="Qwen/Qwen2.5-7B-Instruct",
    output_dir=qwen_output,
    train_df=df_train,
    val_df=df_val,
    max_length=1024,
    num_epochs=2,
    lr=2e-4,
)

In [ ]:
# Load the 800 evaluation essays WITH profile_text
profile_path = f"{project_dir}/DRESs_800_sampled_with_profiles.csv"
df_eval = pd.read_csv(profile_path)

assert "profile_text" in df_eval.columns
assert len(df_eval) == 800

df_eval.head()


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch
import re
from tqdm.auto import tqdm

# Fine-tuned LLaMA adapter path
llama_ft_dir = f"{project_dir}/llama2_dress_scoring_lora"
llama_base_name = "meta-llama/Llama-2-7b-chat-hf"

# Load 4-bit base model
bnb_config_llama = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer_llama_ft = AutoTokenizer.from_pretrained(llama_ft_dir)
if tokenizer_llama_ft.pad_token is None:
    tokenizer_llama_ft.pad_token = tokenizer_llama_ft.eos_token

base_llama_model = AutoModelForCausalLM.from_pretrained(
    llama_base_name,
    quantization_config=bnb_config_llama,
    device_map="auto",
)

# Attach LoRA fine-tuned weights
llama_ft_model = PeftModel.from_pretrained(base_llama_model, llama_ft_dir)
llama_ft_model.eval()

print("Loaded fine-tuned LLaMA-2 on:", llama_ft_model.device)

def get_llama_ft_score(text):
    messages = [
        {"role": "system",
         "content": "You are an automatic writing scoring system. "
                    "Return ONLY a number from 1 to 5."},
        {"role": "user",
         "content": text + "\n\nFINAL ANSWER FORMAT:\n<score>\n"}
    ]

    prompt = tokenizer_llama_ft.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    inputs = tokenizer_llama_ft(prompt, return_tensors="pt",
                                truncation=True, max_length=2048).to(llama_ft_model.device)

    input_len = inputs["input_ids"].shape[1]

    with torch.no_grad():
        output = llama_ft_model.generate(
            **inputs, max_new_tokens=6, do_sample=False
        )

    new_tokens = output[0][input_len:]
    decoded = tokenizer_llama_ft.decode(new_tokens, skip_special_tokens=True).strip()

    m = re.search(r"\d+(\.\d+)?", decoded)
    return float(m.group(0)) if m else None


In [ ]:
tqdm.pandas(desc="Scoring 800 essays with fine-tuned LLaMA-2")

df_eval["llm_score_llama_ft"] = df_eval["profile_text"].progress_apply(
    get_llama_ft_score
)

# Save FT LLaMA scoring CSV
llama_ft_scores_path = f"{project_dir}/DRESs_800_llama_ft_scores.csv"
df_eval.to_csv(llama_ft_scores_path, index=False)

print("Saved fine-tuned LLaMA scores:", llama_ft_scores_path)


In [ ]:
# Fine-tuned Qwen adapter path
qwen_ft_dir = f"{project_dir}/qwen2_5_7b_dress_scoring_lora"
qwen_base_name = "Qwen/Qwen2.5-7B-Instruct"

bnb_config_qwen = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Load tokenizer + base model
tokenizer_qwen_ft = AutoTokenizer.from_pretrained(qwen_ft_dir, use_fast=False)
if tokenizer_qwen_ft.pad_token is None:
    tokenizer_qwen_ft.pad_token = tokenizer_qwen_ft.eos_token

base_qwen_model = AutoModelForCausalLM.from_pretrained(
    qwen_base_name,
    quantization_config=bnb_config_qwen,
    device_map="auto",
)

# Attach LoRA adapter
qwen_ft_model = PeftModel.from_pretrained(base_qwen_model, qwen_ft_dir)
qwen_ft_model.eval()

print("Loaded fine-tuned Qwen on:", qwen_ft_model.device)


def get_qwen_ft_score(text):
    prompt = (
        "You are an automatic writing scoring system. "
        "Return ONLY a number between 1 and 5.\n\n"
        f"{text}\n\nFinal score:"
    )

    inputs = tokenizer_qwen_ft(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=2048
    ).to(qwen_ft_model.device)

    input_len = inputs["input_ids"].shape[1]

    with torch.no_grad():
        output = qwen_ft_model.generate(
            **inputs, max_new_tokens=6, do_sample=False
        )

    new_tokens = output[0][input_len:]
    decoded = tokenizer_qwen_ft.decode(new_tokens, skip_special_tokens=True).strip()

    m = re.search(r"\d+(\.\d+)?", decoded)
    return float(m.group(0)) if m else None


In [ ]:
df_eval_qwen = pd.read_csv(profile_path)

tqdm.pandas(desc="Scoring 800 essays with fine-tuned Qwen")

df_eval_qwen["llm_score_qwen_ft"] = df_eval_qwen["profile_text"].progress_apply(
    get_qwen_ft_score
)

qwen_ft_scores_path = f"{project_dir}/DRESs_800_qwen_ft_scores.csv"
df_eval_qwen.to_csv(qwen_ft_scores_path, index=False)

print("Saved fine-tuned Qwen scores:", qwen_ft_scores_path)


In [ ]:
base_llama_path = f"{project_dir}/DRESs_800_llama_scores_base.csv"
base_qwen_path  = f"{project_dir}/DRESs_800_qwen_scores_base_local.csv"

df_llama_base = pd.read_csv(base_llama_path)
df_qwen_base  = pd.read_csv(base_qwen_path)

print("Loaded base LLaMA shape:", df_llama_base.shape)
print("Loaded base Qwen shape:", df_qwen_base.shape)


In [ ]:
# Use _key to align rows
key_cols = ["prompt", "essay", "total", "content", "organization", "language"]

def make_key(df):
    df["_key"] = df[key_cols].astype(str).agg(" | ".join, axis=1)
    return df

df_llama_base = make_key(df_llama_base)
df_qwen_base  = make_key(df_qwen_base)
df_llama_ft   = make_key(pd.read_csv(llama_ft_scores_path))
df_qwen_ft    = make_key(pd.read_csv(qwen_ft_scores_path))

# Merge all scores into one comparison table
merged = df_llama_base[["_key","group","profile_text","total","llm_score_llama_base"]] \
    .merge(df_llama_ft[["_key","llm_score_llama_ft"]], on="_key") \
    .merge(df_qwen_base[["_key","llm_score_qwen_base"]], on="_key") \
    .merge(df_qwen_ft[["_key","llm_score_qwen_ft"]], on="_key")

merged.head()


In [ ]:
combined_path = f"{project_dir}/DRESs_800_scores_base_and_ft.csv"
merged.to_csv(combined_path, index=False)

print("Saved merged base + FT comparison to:", combined_path)


In [ ]:
df_ft = merged.copy()

# selection flags for FT models
LLM_SELECT_THRESHOLD = 3.5
df_ft["selected_llama_ft"] = df_ft["llm_score_llama_ft"] >= LLM_SELECT_THRESHOLD
df_ft["selected_qwen_ft"]  = df_ft["llm_score_qwen_ft"]  >= LLM_SELECT_THRESHOLD

# qualified flag
df_ft["qualified"] = df_ft["total"] >= 10.0

def compute_rabbi(df, group_col, score_col, g1, g2):
    a = df[df[group_col]==g1][score_col].dropna().values
    b = df[df[group_col]==g2][score_col].dropna().values
    if len(a)==0 or len(b)==0:
        return np.nan
    fav_a = (a[:,None] > b[None,:]).sum()
    fav_b = (a[:,None] < b[None,:]).sum()
    return (fav_a - fav_b) / (len(a)*len(b))

def compute_dp(df, group_col, select_col, g1, g2):
    return float(df[df[group_col]==g1][select_col].mean() -
                 df[df[group_col]==g2][select_col].mean())

def compute_eo(df, group_col, select_col, qualified_col, g1, g2):
    df_q = df[df[qualified_col]]
    return float(df_q[df_q[group_col]==g1][select_col].mean() -
                 df_q[df_q[group_col]==g2][select_col].mean())

def delta_distance(df, group_col, score_col, g1, g2):
    a = df[df[group_col]==g1][score_col].dropna()
    b = df[df[group_col]==g2][score_col].dropna()
    return float(abs(a.mean() - b.mean()))

from scipy.spatial.distance import jensenshannon
from scipy.stats import wasserstein_distance

def compute_jsd(df, group_col, score_col, g1, g2, bins=20):
    a = df[df[group_col]==g1][score_col].dropna().values
    b = df[df[group_col]==g2][score_col].dropna().values
    if len(a)==0 or len(b)==0:
        return np.nan
    ha,_ = np.histogram(a, bins=bins, range=(1,5), density=True)
    hb,_ = np.histogram(b, bins=bins, range=(1,5), density=True)
    return float(jensenshannon(ha+1e-12, hb+1e-12))

def compute_emd(df, group_col, score_col, g1, g2):
    a = df[df[group_col]==g1][score_col].dropna().values
    b = df[df[group_col]==g2][score_col].dropna().values
    if len(a)==0 or len(b)==0:
        return np.nan
    return float(wasserstein_distance(a, b))


metrics_ft = []
groups = sorted(df_ft["group"].unique())
BASE_GROUP = "Undergrad Male"

def add_model_rows(df, model_name, score_col, select_col):
    rows = []
    for g in groups:
        rows.append({
            "model": model_name,
            "group": g,
            "ΔDP": compute_dp(df, "group", select_col, g, BASE_GROUP),
            "ΔEO": compute_eo(df, "group", select_col, "qualified", g, BASE_GROUP),
            "RABBI_DP": compute_rabbi(df, "group", score_col, g, BASE_GROUP),
            "RABBI_EO": compute_rabbi(df[df["qualified"]], "group", score_col, g, BASE_GROUP),
            "δ": delta_distance(df, "group", score_col, g, BASE_GROUP),
            "JSD": compute_jsd(df, "group", score_col, g, BASE_GROUP),
            "EMD": compute_emd(df, "group", score_col, g, BASE_GROUP),
        })
    return rows

metrics_ft += add_model_rows(df_ft, "Llama-2-7B-FT", "llm_score_llama_ft", "selected_llama_ft")
metrics_ft += add_model_rows(df_ft, "Qwen2.5-7B-FT", "llm_score_qwen_ft", "selected_qwen_ft")

results_ft_df = pd.DataFrame(metrics_ft)
results_ft_df

In [ ]:
metrics = ["ΔDP","ΔEO","RABBI_DP","RABBI_EO","δ","JSD","EMD"]
groups  = sorted(results_ft_df["group"].unique())
models  = sorted(results_ft_df["model"].unique())

fig, axes = plt.subplots(2, 4, figsize=(26,12))
axes = axes.flatten()

for i, metric in enumerate(metrics):
    ax = axes[i]
    pivot = results_ft_df.pivot(index="group", columns="model", values=metric).loc[groups]
    pivot.plot(kind="bar", ax=ax)
    ax.set_title(metric, fontsize=14)
    ax.axhline(0, color="black")
    ax.set_xlabel("Group")
    ax.set_ylabel(metric)
    ax.tick_params(axis='x', rotation=20)

fig.delaxes(axes[-1])
plt.tight_layout()

save_path = f"{project_dir}/fairness_metrics_finetuned_subplots.png"
fig.savefig(save_path, dpi=300, bbox_inches='tight')
plt.show()
print("Saved subplot figure:", save_path)

In [ ]:
metrics = ["ΔDP","ΔEO","RABBI_DP","RABBI_EO","δ","JSD","EMD"]
models  = sorted(results_ft_df["model"].unique())
groups  = sorted(results_ft_df["group"].unique())

max_val = max(results_ft_df[m].abs().max() for m in metrics)

fig, axes = plt.subplots(1, len(models), figsize=(6*len(models),6), sharey=True)

if len(models)==1:
    axes=[axes]

heatmaps=[]

for ax, m in zip(axes, models):
    sub = results_ft_df[results_ft_df["model"]==m]

    heat_data = np.array([[abs(sub[sub["group"]==g][metric]).iloc[0]
                           for metric in metrics]
                           for g in groups])
    text_data = np.array([[sub[sub["group"]==g][metric].iloc[0]
                           for metric in metrics]
                           for g in groups])

    im = ax.imshow(heat_data, cmap="YlOrRd", vmin=0, vmax=max_val, aspect="auto")
    heatmaps.append(im)

    ax.set_xticks(np.arange(len(metrics)))
    ax.set_xticklabels(metrics, rotation=45)
    ax.set_yticks(np.arange(len(groups)))
    ax.set_yticklabels(groups)
    ax.set_title(m)

    for i in range(len(groups)):
        for j in range(len(metrics)):
            val = text_data[i,j]
            ax.text(j, i, f"{val:.2f}",
                    ha="center", va="center",
                    color="white" if heat_data[i,j] > max_val*0.5 else "black")

cbar = fig.colorbar(heatmaps[0], ax=axes, orientation="horizontal", fraction=0.05, pad=0.12)
cbar.set_label("Magnitude (|value|)")

save_path = f"{project_dir}/fairness_heatmap_finetuned_combined.png"
fig.savefig(save_path, dpi=400)
plt.show()
print("Saved heatmap:", save_path)